In [1]:
# import numpy
# import pandas as pd
# import matplotlib.pyplot as plt
# import math
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error

# import tensorflow as tf 
# import keras

In [2]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices: 
#     tf.config.experimental.set_memory_growth(device, True)
#     print(device)

In [3]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [4]:
# config = tf.compat.v1.ConfigProto( device_count = {'GPU': 0 , 'GPU': 1} ) 
# sess = tf.compat.v1.Session(config=config) 
# keras.backend.set_session(sess)

In [5]:
import pandas as pd
import numpy as np

In [6]:
#顯示所有列
pd.set_option('display.max_columns', None)

In [7]:
# Reading the data
train = pd.read_csv('tbrain_cc_training_48tags_hash_final.csv')
print(train.shape)
print(train.info())
train.head(5)

(32975653, 53)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32975653 entries, 0 to 32975652
Data columns (total 53 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int64  
 1   chid                      int64  
 2   shop_tag                  object 
 3   txn_cnt                   int64  
 4   txn_amt                   float64
 5   domestic_offline_cnt      int64  
 6   domestic_online_cnt       int64  
 7   overseas_offline_cnt      int64  
 8   overseas_online_cnt       int64  
 9   domestic_offline_amt_pct  float64
 10  domestic_online_amt_pct   float64
 11  overseas_offline_amt_pct  float64
 12  overseas_online_amt_pct   float64
 13  card_1_txn_cnt            int64  
 14  card_2_txn_cnt            int64  
 15  card_3_txn_cnt            int64  
 16  card_4_txn_cnt            int64  
 17  card_5_txn_cnt            int64  
 18  card_6_txn_cnt            int64  
 19  card_7_txn_cnt            int64  
 20  card_8_

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10321418,45,3,3891.965283,3,0,0,0,1.00,0.00,0.0,0.0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1
1,1,10414574,15,2,10616.561549,2,0,0,0,1.00,0.00,0.0,0.0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1
2,1,10134567,48,2,23527.655416,0,2,0,0,0.00,1.00,0.0,0.0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1
3,1,10001003,48,9,17751.558260,7,2,0,0,0.75,0.25,0.0,0.0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1
4,1,10267183,2,1,21701.307598,1,0,0,0,1.00,0.00,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0


In [8]:
predict = pd.DataFrame(pd.read_csv('需預測的顧客名單及提交檔案範例.csv'))
predict = predict.drop(columns = ['top1','top2','top3'])
print(predict.shape)
print(predict.info())
predict.head(5)

(500000, 1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   chid    500000 non-null  int64
dtypes: int64(1)
memory usage: 3.8 MB
None


,chid
0,10128239
1,10077943
2,10277876
3,10364842
4,10392717


In [9]:
# train空值往前填充
train.fillna(method='ffill', inplace=True)

In [10]:
# 把shop_tag的other取代為49
train['shop_tag']= train['shop_tag'].replace('other','49')

In [11]:
# 把shop_tag的object型態轉換為數字
train["shop_tag"] = train["shop_tag"].astype(str).astype(int)

In [12]:
# 對txn_amt、slam做Normalization，因為小數位數過大會報錯
train['txn_amt'] = (train['txn_amt'] - train['txn_amt'].min())/\
                            (train['txn_amt'].max() - train['txn_amt'].min())
train['slam'] = (train['slam'] - train['slam'].min())/\
                            (train['slam'].max() - train['slam'].min())

In [13]:
# 對卡片消費次數做加總，並刪掉個別消費次數，這邊可能要再試
train["sum_card_txn_cnt"] = train["card_1_txn_cnt"] + train["card_2_txn_cnt"] + train["card_3_txn_cnt"] + train["card_4_txn_cnt"] + train["card_5_txn_cnt"] + train["card_6_txn_cnt"] + train["card_7_txn_cnt"] + train["card_8_txn_cnt"] + train["card_9_txn_cnt"] + train["card_10_txn_cnt"] + train["card_11_txn_cnt"] + train["card_12_txn_cnt"] + train["card_13_txn_cnt"] + train["card_14_txn_cnt"] + train["card_other_txn_cnt"]
train = train.drop(columns = ['card_1_txn_cnt','card_2_txn_cnt','card_3_txn_cnt','card_4_txn_cnt','card_5_txn_cnt','card_6_txn_cnt','card_7_txn_cnt','card_8_txn_cnt','card_9_txn_cnt','card_10_txn_cnt','card_11_txn_cnt','card_12_txn_cnt','card_13_txn_cnt','card_14_txn_cnt','card_other_txn_cnt'])

# # 對卡片消費次數做加總，不刪掉個別次數，這邊可能要再試
# train["sum_card_txn_cnt"] = train["card_1_txn_cnt"] + train["card_2_txn_cnt"] + train["card_3_txn_cnt"] + train["card_4_txn_cnt"] + train["card_5_txn_cnt"] + train["card_6_txn_cnt"] + train["card_7_txn_cnt"] + train["card_8_txn_cnt"] + train["card_9_txn_cnt"] + train["card_10_txn_cnt"] + train["card_11_txn_cnt"] + train["card_12_txn_cnt"] + train["card_13_txn_cnt"] + train["card_14_txn_cnt"] + train["card_other_txn_cnt"]

In [14]:
train.shop_tag.value_counts()

37    4092401
49    2988218
15    2573145
36    2550500
45    2411310
10    2234101
2     1971956
48    1806496
12    1130825
19    1022234
25     795763
6      634849
18     629407
16     628987
13     625801
31     607888
22     547293
39     535248
21     503873
26     476687
46     424055
9      349932
17     345814
11     280455
33     261088
5      231969
47     225170
32     202863
35     185585
34     183470
41     166662
7      160780
20     130038
8      129061
23     124889
40     118913
43     114998
38     113608
3      109286
42      91492
14      73457
30      62032
44      55273
24      31530
1       20800
4        5997
27       4754
28       4585
29        115
Name: shop_tag, dtype: int64

In [15]:
# predict_test是之後要預測並提交上傳的資料，在這邊先拆分出來 抓倒數幾個月中消費次數最多的時候根據chid做刪除重複項的動作
# train同時也做刪減動作
# predict_test = train.iloc[train[(train['dt']==24) | (train['dt']==23) | (train['dt']==22) | (train['dt']==21) | (train['dt']==20) | (train['dt']==19) | (train['dt']==18) | (train['dt']==17) | (train['dt']==16) | (train['dt']==15) | (train['dt']==14) | (train['dt']==13)].groupby(['chid']).apply(lambda x: x['txn_cnt'].idxmax())]
predict_test = train.iloc[train[(train['dt']==24) | (train['dt']==23) | (train['dt']==22) | (train['dt']==21) | (train['dt']==20) | (train['dt']==19) | (train['dt']==18) | (train['dt']==17) | (train['dt']==16) | (train['dt']==15) | (train['dt']==14) | (train['dt']==13)].groupby(['chid']).apply(lambda x: x['txn_cnt'].idxmax())]

In [16]:
predict_test.head(5)

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,sum_card_txn_cnt
16536056,14,10000000,10,4,1.873943e-111,0,4,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1,4
21082844,17,10000001,37,6,1.428726e-111,6,0,0,0,1.0,0.0,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,11.0,1.0,3.0,30.0,8.670854e-214,0.0,5.0,1,6
14936013,13,10000002,49,1,1.605622e-111,0,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,13.0,1.0,99.0,30.0,1.129543e-213,0.0,3.0,1,1
27393689,21,10000003,37,2,1.205646e-111,2,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,6.0,15.0,1.0,99.0,30.0,1.129543e-213,0.0,6.0,1,2
20388560,17,10000004,49,2,1.106555e-111,1,0,0,1,0.6,0.0,0.0,0.4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,15.0,1.0,99.0,30.0,6.369410e-214,1.0,2.0,1,2


In [17]:
len(predict_test)

500000

In [18]:
len(train)
print(train.shape)
print(train.info())

(32975653, 39)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32975653 entries, 0 to 32975652
Data columns (total 39 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int64  
 1   chid                      int64  
 2   shop_tag                  int64  
 3   txn_cnt                   int64  
 4   txn_amt                   float64
 5   domestic_offline_cnt      int64  
 6   domestic_online_cnt       int64  
 7   overseas_offline_cnt      int64  
 8   overseas_online_cnt       int64  
 9   domestic_offline_amt_pct  float64
 10  domestic_online_amt_pct   float64
 11  overseas_offline_amt_pct  float64
 12  overseas_online_amt_pct   float64
 13  card_1_txn_amt_pct        float64
 14  card_2_txn_amt_pct        float64
 15  card_3_txn_amt_pct        float64
 16  card_4_txn_amt_pct        float64
 17  card_5_txn_amt_pct        float64
 18  card_6_txn_amt_pct        float64
 19  card_7_txn_amt_pct        float64
 20  card_8_

In [19]:
train = train[(train['dt']==24) | (train['dt']==23) | (train['dt']==22) | (train['dt']==21) | (train['dt']==20) | (train['dt']==19) | (train['dt']==18) | (train['dt']==17) | (train['dt']==16) | (train['dt']==15) | (train['dt']==14) | (train['dt']==13)]

In [20]:
len(train)

18454166

In [21]:
# 按照要提交的檔案排序
predict_test = predict_test.set_index('chid', drop= False)
predict_test = predict_test.loc[predict.chid]
predict_test.head(5)

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,sum_card_txn_cnt
chid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10128239,18,10128239,49,2,1.086396e-111,2,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1,2
10077943,13,10077943,13,16,1.888839e-111,0,0,0,16,0.00,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,12.0,1.0,2.0,30.0,7.594606e-214,0.0,5.0,1,16
10277876,15,10277876,37,7,1.904996e-111,4,3,0,0,0.51,0.49,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,6.0,1.0,1.0,30.0,6.083676e-214,1.0,2.0,1,7
10364842,14,10364842,37,7,2.523124e-111,7,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,7.0,1.0,99.0,30.0,1.048096e-213,1.0,4.0,1,7
10392717,13,10392717,48,9,2.320924e-111,9,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,5.0,1.0,2.0,30.0,3.248696e-214,1.0,2.0,1,9


In [22]:
train.columns

Index(['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
       'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
       'gender_code', 'age', 'primary_card', 'sum_card_txn_cnt'],
      dtype='object')

In [23]:
train.head(5)

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,sum_card_txn_cnt
14521362,13,10115966,15,1,1.474382e-111,1,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,15.0,1.0,1.0,30.0,1.980761e-213,0.0,5.0,1,1
14521370,13,10115966,18,3,2.915390e-111,1,2,0,0,0.23,0.77,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,15.0,1.0,1.0,30.0,1.980761e-213,0.0,5.0,1,3
14521381,13,10115966,21,1,1.466423e-111,1,0,0,0,1.00,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,15.0,1.0,1.0,30.0,1.980761e-213,0.0,5.0,1,1
14521391,13,10115966,37,8,2.580079e-111,1,7,0,0,0.12,0.88,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,15.0,1.0,1.0,30.0,1.980761e-213,0.0,5.0,1,8
14521400,13,10294163,36,1,8.665268e-112,1,0,0,0,1.00,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,11.0,1.0,2.0,30.0,9.594880e-214,0.0,3.0,1,1


In [24]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [25]:
target_column_train = ['shop_tag']
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

(14763332, 38)
(14763332, 1)


In [26]:
target_column_test = ['shop_tag'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

(3690834, 38)
(3690834, 1)


In [27]:
# 建立 XGBClassifier 模型
from xgboost import XGBClassifier
xgboostModel = XGBClassifier(random_state=42)
# clf = XGBClassifier(
#         #樹的個數
#         n_estimators=100,
#         # 如同學習率
#         learning_rate= 0.3, 
#         # 構建樹的深度，越大越容易過擬合    
#         max_depth=1, 
#         # 隨機取樣訓練樣本 訓練例項的子取樣比
#         subsample=1, 
#         # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
#         gamma=0, 
#         # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
#         reg_lambda=1,  
        
#         #最大增量步長，我們允許每個樹的權重估計。
#         max_delta_step=0,
#         # 生成樹時進行的列取樣 
#         colsample_bytree=1, 

#         # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
#         # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
#         #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
#         min_child_weight=1, 

#         #隨機種子
#         seed=42
        
#         # L1 正則項引數
# #        reg_alpha=0,
        
#         #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重
#         #scale_pos_weight=1,
        
#         #多分類的問題 指定學習任務和相應的學習目標
#         #objective= 'multi:softmax', 
        
#         # 類別數，多分類與 multisoftmax 並用
#         #num_class=10,
        
#         # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
# #        silent=0 ,
#         # cpu 執行緒數 預設最大
# #        nthread=4,
    
#         #eval_metric= 'auc'
# )



In [28]:
# 使用訓練資料訓練模型
xgboostModel.fit(X_train[0:2000000], y_train[0:2000000])

/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to 

[21:00:54] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [29]:
# pre_Train = predict_test.drop(columns = ['shop_tag', 'chid'], axis=1, inplace=True)

In [30]:
predict_test.drop(columns = ['shop_tag'], axis=1, inplace=True)

In [31]:
# 模型預測
predictions = xgboostModel.predict_proba(predict_test)

In [32]:
best_predictions = np.argsort(-predictions, axis=1)[:, :49]
best_predictions[0]

array([48, 11, 12, 47, 34, 14, 20, 42,  5,  1, 45, 38, 18,  8, 41, 29,  9,
       46, 10, 32, 36, 25, 44, 22,  6, 43,  2, 17, 24, 40, 37, 21, 35, 13,
       15, 31,  4, 30, 39, 16,  7, 23, 33, 26,  0, 19,  3, 28, 27])

In [33]:
best_predictions_final = np.zeros((len(best_predictions), 3), dtype=int)

In [34]:
for a in range(len(best_predictions)):
    sum = 0
    for i in range(49): 
        if (best_predictions[a, i] == [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]).any():
            best_predictions_final[a, sum] = int(best_predictions[a, i])
            sum += 1 
        if sum == 3: 
            break

In [35]:
best_predictions_final

array([[48, 12, 18],
       [48, 18, 25],
       [48, 22, 12],
       ...,
       [48, 12, 18],
       [48, 12, 18],
       [48, 12, 18]])

In [36]:
submission = pd.DataFrame({ 'chid':predict.chid,'top1':best_predictions_final[0:len(best_predictions_final),0] , 'top2':best_predictions_final[0:len(best_predictions_final),1] , 'top3':best_predictions_final[0:len(best_predictions_final),2] })

In [37]:
filename = 'submission.csv'
submission.to_csv(filename,index=False)
print(submission)
print('Saved file: ' + filename)

            chid  top1  top2  top3
0       10128239    48    12    18
1       10077943    48    18    25
2       10277876    48    22    12
3       10364842    48    12    18
4       10392717    48    12    18
...          ...   ...   ...   ...
499995  10433239    48    18    22
499996  10235119    48    22    18
499997  10267595    48    12    18
499998  10160606    48    12    18
499999  10141759    48    12    18

[500000 rows x 4 columns]
Saved file: submission.csv
